In [ ]:
#!sshpass -p 'PASSWORD' rsync -a ssrl:/mnt/data00/b_sli/bl1-5/June15SSRL/samples/ . -v

In [ ]:
import warnings
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.patches import Circle
import numpy as np
import numpy.ma as ma
import scipy.signal as ss
import skimage.morphology as skm
import skimage.transform as skt
from PIL import Image
from tqdm.notebook import tqdm

##SETTINGS###
E = 10  # keV
detz = 2880  # mm
pixelsize = 0.079  # mm
centerx = 1025  # found using irena with tilts set to zero for silver behenate calibration
centery = 1015
emptyfiddle = 0  # factor for empty frame subtraction. 0: dont do empty subtraction

path = Path("/Users/thunder/msc/ssrl/june2021/samples")
maskname = "maskPS_ox11_uneven_notilt_posx_-9_poxy-32_0615180549_0001.tif"  # masking only the BS
emptyname = "xrayonly_posx_-9.7_posy_-32_0615131048_0002.tif"
########

mask = skm.binary_opening(np.array(Image.open(path / maskname)), skm.disk(3))
empty = ma.array(Image.open(path / emptyname), float, mask=mask == 0)
qstep = pixelsize / detz * (2 * np.pi) / (1.24 / E)

In [ ]:
def radial_profile(data, center=None, calcStd=False, os=1):
    """
    calculates a ND radial profile of data around center. will ignore nans
    calStd: calculate standard error, return tuple of (profile, stderr)
    os: oversample by a factor. With default 1 the stepsize will be 1 pixel, with 2 it will be .5 pixels etc.
    """
    import numpy as _np
    with _np.errstate(invalid='ignore',divide='ignore'):
        if center is None:
            center = _np.array(data.shape) // 2
        if len(center) != data.ndim:
            raise TypeError("center should be of length data.ndim")
        center = _np.array(center)[tuple([slice(len(center))] + data.ndim * [None])]
        ind = _np.indices(data.shape)
        r = (_np.rint(os * _np.sqrt(((ind - center) ** 2).sum(axis=0)))).astype(int)
        databin = _np.bincount(r.ravel(), (_np.nan_to_num(data)).ravel())
        nr = _np.bincount(r.ravel(), ((~_np.isnan(data)).astype(float)).ravel())
        radialprofile = databin / nr
        if not calcStd:
            return radialprofile

        data2bin = _np.bincount(r.ravel(), (_np.nan_to_num(data ** 2)).ravel())
        radial2profile = data2bin / nr
        std = _np.sqrt(radial2profile - radialprofile ** 2)/np.sqrt(nr)
        return radialprofile, std

In [ ]:
# plot only useful images..
fns = sorted(
    filter(lambda x: 
           ("sul" in str(x) or "ox" in str(x)) 
           and not ("mask" in str(x) or "only" in str(x)) 
           and "_0001" in str(x), path.glob("*.tif"))
)

In [ ]:
with PdfPages("result.pdf") as pdf:
    for fn in tqdm(fns):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            warnings.simplefilter("ignore", category=UserWarning)

            im = np.array(Image.open(fn), float)
            mim = ma.array(im, mask=mask == 0) - emptyfiddle * empty
            mim[mim < 0] = ma.masked

            f, ((axsaxs, axrad), (axpolar, axlines)) = plt.subplots(2, 2, figsize=(20, 20))
            
            #plot saxs
            axsaxs.matshow(
                np.log10(mim),
                vmin=np.percentile(np.log10(mim[512:-512]), 10),
                vmax=np.percentile(np.log10(mim[512:-512]), 99.99),
                extent=[qstep * x for x in (-centery, 2048 - centery, -2048 + centerx, centerx)],
            )
            axsaxs.set_xlim(-0.4, 0.4)
            axsaxs.set_ylim(-0.4, 0.4)
            axsaxs.set_xlabel("$q_{hor} / nm^{-1}$")
            axsaxs.set_ylabel("$q_{vert} / nm^{-1}$")
            for r in [0.05, 0.1, 0.2, 0.3]:
                circ = Circle(
                    (0, 0),
                    r,
                    fill=False,
                    linestyle=":",
                    alpha=0.5,
                )
                axsaxs.add_patch(circ)
                axsaxs.text(0, r, f'{r}{"$nm^{-1}$"}', fontsize="x-small")
            
            #radial profile
            r, err = radial_profile(mim.filled(np.nan), center=(centerx, centery), calcStd=True)[:800]

            peaks = ss.find_peaks(np.log10(r[:250]), distance=10, prominence=0.01)
            axrad.errorbar(
                np.arange(len(r)) * qstep,
                r,
                yerr=2 * err,
                label="radial mean $\pm 2stderr$",
            )
            axrad.set_xscale("log")
            axrad.set_yscale("log")

            axrad.legend()
            for p in peaks[0]:
                qpos = (p) * qstep
                axrad.text(qpos, r[p], f"{qpos:.2f}")
            axrad.set_xlabel("$q/nm^{-1}$")
            
            #polar projection
            polar = skt.warp_polar(
                mim.filled(np.nan),
                center=(centerx, centery),
                output_shape=(360, 512),
                radius=512,
            )
            axpolar.matshow(
                np.log10(polar),
                vmax=np.nanpercentile(np.log10(polar), 99.9),
                vmin=np.nanpercentile(np.log10(polar), 1),
            )
            axpolar.set_xticklabels([f"{p*qstep:.2f}" for p in axpolar.get_xticks()])
            axpolar.set_yticklabels([f"{int(p)}°" for p in axpolar.get_yticks()])

            axpolar.set_xlabel("q/$nm^{-1}$")
            axpolar.set_ylabel("$\phi$")
            
            #line profiles
            angular = np.nanmean((polar[:, :60]), 1)
            angularpeaks = ss.find_peaks(np.log10(angular), distance=10, prominence=0.12)

            for p in angularpeaks[0]: #streaks
                axpolar.axhline(p, linestyle=":")
                axpolar.text(0, p, f"{p:.2f}")
                prof = np.nanmean(polar[p - 2 : p + 3, :200], 0)
                axlines.loglog(np.arange(len(prof)) * qstep, prof, ":", label=f"{p}°")

            for p0, p1 in zip(angularpeaks[0], np.roll(angularpeaks[0], -1)): #in between streaks
                p = ((p1 + p0) / 2) % 360 if p1 > p0 else ((p1 - (360 - p0)) / 2) % 360
                prof = np.nanmean(polar[int(p) - 2 : int(p) + 3, :200], 0)
                axlines.loglog(np.arange(len(prof)) * qstep, prof, label=f"{p}°")

                axpolar.axhline(p, c="r")

            axlines.set_xlabel("q/$nm^{-1}$")
            axlines.legend(title="line profiles at $\phi$")
            f.suptitle(fn.stem)
            f.tight_layout(rect=[0, 0.01, 1, 0.98])
            pdf.savefig(f)
            plt.show()